<a href="https://colab.research.google.com/github/yorkjong/Colab/blob/main/bitfinex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#@title Ticker { run: "auto", display-mode: "form" }
currency = "USD" #@param ["USD", "UST", "BTC", "ETH", "SOL", "DOGE"] {allow-input: true}

import requests
import json
import pandas as pd

symbol = f'f{currency}'
url = f'https://api-pub.bitfinex.com/v2/ticker/{symbol}'
response = requests.get(url)
data = response.json()

headers = ['FRR', 'BID (%)', 'BID_PERIOD', 'BID_SIZE', 'ASK (%)', 'ASK_PERIOD', 'ASK_SIZE', 'DAILY_CHANGE', 'DAILY_CHANGE_PERC', 'LAST_PRICE (%)', 'VOLUME', 'HIGH (%)', 'LOW (%)', '?', '??', 'FRR_AMOUNT_AVAILABLE']
df = pd.DataFrame([data], columns=headers)
df = df.drop(columns=['FRR', 'DAILY_CHANGE', 'DAILY_CHANGE_PERC', '?', '??', 'FRR_AMOUNT_AVAILABLE'])
for name in ('BID (%)', 'ASK (%)', 'LAST_PRICE (%)', 'HIGH (%)', 'LOW (%)'):
    df[name] = [r*100 for r in df[name]]

display(df)

,BID (%),BID_PERIOD,BID_SIZE,ASK (%),ASK_PERIOD,ASK_SIZE,LAST_PRICE (%),VOLUME,HIGH (%),LOW (%)
0,0.0376,120,1.232090e+07,0.028713,2,149337.03426,0.029333,2.518931e+08,0.075045,0.000985


In [50]:
#@title Candles { run: "auto", display-mode: "form" }
currency = "USD" #@param ["USD", "UST", "BTC", "ETH", "SOL", "DOGE"] {allow-input: true}
interval = "1D" #@param ["1m", "5m", "15m", "30m", "1h", "3h", "6h", "12h", "1D", "1W", "14D", "1M"]

import requests
import json
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

period = 30 
candle = f'trade:{interval}:f{currency}:p{period}'
section = 'hist'    # last, hist
url = f"https://api-pub.bitfinex.com/v2/candles/{candle}/{section}"
response = requests.get(url)
data = response.json()

headers = ['MTS', 'OPEN', 'CLOSE', 'HIGH', 'LOW', 'VOLUME']
df = pd.DataFrame(data, columns=headers)
df = df.loc[::-1].reset_index(drop=True)    # Reverse Row
for name in ('OPEN', 'CLOSE', 'HIGH', 'LOW'):
    df[name] = [r*100 for r in df[name]]

# Initialize empty plot with a marginal subplot
fig = make_subplots(
    rows=2, cols=1, 
    row_heights=[0.7, 0.3],
    vertical_spacing=0.03, 
    figure=go.Figure(layout=go.Layout(height=720))
)

# Plot the candlestick chart
candlestick = go.Candlestick(
    x=df.index, 
    open=df['OPEN'], high=df['HIGH'], low=df['LOW'], close=df['CLOSE'], 
    name='OHLC'
)
fig.add_trace(candlestick)

# Add moving averages to the figure
ma_days = (5, 10, 20, 50)
ma_colors = ('orange', 'red', 'green', 'blue', 'brown')
for d, c in zip(ma_days, ma_colors):
    df[f'ma{d}'] = df['CLOSE'].rolling(window=d).mean()
    ma = go.Scatter(x=df.index, y=df[f'ma{d}'], name=f'MA {d}', 
                    line=dict(color=f'{c}', width=2), opacity=0.4)
    fig.add_trace(ma)

# Add volume trace to 2nd row
colors = ['green' if o - c >= 0
          else 'red' for o, c in zip(df['OPEN'], df['CLOSE'])]
volume = go.Bar(x=df.index, y=df['VOLUME'], name='Volume', 
                marker_color=colors, opacity=0.5)
fig.add_trace(volume, row=2, col=1)

# Add moving average volume to 2nd row
df['vma50'] = df['VOLUME'].rolling(window=50).mean()
vma50 = go.Scatter(x=df.index, y=df['vma50'], name='VMA 50', 
                   line=dict(color='purple', width=2))
fig.add_trace(vma50, row=2, col=1)

# Update layout
fig.update_layout(
    title=f'{currency}', 
    title_x=0.5, title_y=.9,
    
    xaxis=dict(anchor='free'),
    yaxis=dict(anchor='x2', side='right', title='Founding Rate (%)'),  
    yaxis2=dict(anchor='x', side='right', title='Volume'),

    legend=dict(yanchor='middle', y=0.5, xanchor="left", x=0.01),
    xaxis_rangeslider_visible=False,
)

# Add crosshair cursor
fig.update_yaxes(
    spikemode='across', spikesnap='cursor', 
    spikethickness=1, spikedash='solid', spikecolor='grey')
fig.update_xaxes(
    spikemode='across', spikesnap='cursor',
    spikethickness=1, spikedash='solid', spikecolor='grey')
fig.update_layout(hovermode='x')  # 'x', 'y', 'closest', False, 'x unified', 
                                  # 'y unified'

# Add a dropdown menu for selecting a hovermode
hovermodes = ('x', 'y', 'closest', 'x unified', 'y unified')
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(args=[dict(hovermode=m)],
                        label=m, method="relayout") for m in hovermodes
            ]),
            showactive=True,
            xanchor='left', yanchor='bottom',
            x=0, y=1.05
        ),
    ],
    annotations=[dict(
        text="hovermode:", showarrow=False,
        x=0, y=1.15, xref="paper", yref="paper", align="left"
    )],
)

# Show the figure
fig.show()
display(df.tail(10))

,MTS,OPEN,CLOSE,HIGH,LOW,VOLUME,ma5,ma10,ma20,ma50,vma50
110,1680307200000,0.041600,0.078000,0.096350,0.028637,8.602410e+07,0.042117,0.040177,0.041129,0.054541,3.460683e+07
111,1680393600000,0.067125,0.067011,0.088000,0.006700,1.067659e+07,0.049519,0.043879,0.042772,0.054468,3.469114e+07
112,1680480000000,0.067013,0.066760,0.087000,0.030000,1.161104e+07,0.055356,0.047255,0.044581,0.054401,3.445901e+07
113,1680566400000,0.066739,0.032008,0.084383,0.012000,3.483411e+07,0.055356,0.047331,0.044431,0.053636,3.471288e+07
114,1680652800000,0.050000,0.032999,0.085000,0.028944,2.301460e+07,0.055356,0.047430,0.044681,0.052895,3.503901e+07
115,1680739200000,0.031000,0.032999,0.099900,0.031000,7.154792e+08,0.046355,0.044236,0.043093,0.052155,4.907417e+07
116,1680825600000,0.033699,0.061789,0.099900,0.029406,3.915708e+07,0.045311,0.047415,0.043683,0.052915,4.895198e+07
117,1680912000000,0.060789,0.052450,0.090000,0.030000,6.661602e+07,0.042449,0.048902,0.043806,0.052579,4.949882e+07
118,1680998400000,0.032569,0.032999,0.075045,0.010000,9.677427e+07,0.042647,0.049002,0.042266,0.051857,5.103341e+07
119,1681084800000,0.033699,0.055890,0.055890,0.032999,3.675001e+04,0.047225,0.051290,0.043484,0.051605,5.012311e+07


### Reference
Bitfinex API:
- [Public Endpoints](https://docs.bitfinex.com/docs/rest-public)
- [Ticker](https://docs.bitfinex.com/reference/rest-public-ticker)
- [Candles](https://docs.bitfinex.com/reference/rest-public-candles)